In [3]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv("data\data.csv",encoding="latin-1")
df

,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4843,negative,Net sales of the Paper segment decreased to EU...


In [5]:
df.columns

Index(['neutral', 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .'], dtype='object')

In [6]:
df.columns=['sentiment','news']

In [7]:
df

,sentiment,news
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4843,negative,Net sales of the Paper segment decreased to EU...


In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [73]:
df.news

0       technopoli plan develop stage area less squar ...
1       intern electron industri compani elcoteq laid ...
2       new product plant compani would increas capac ...
3       accord compani updat strategi year baswar targ...
4       financ aspocomp growth aspocomp aggress pursu ...
                              ...                        
4840    london marketwatch share price end lower londo...
4841    rinkuskiai beer sale fell per cent million lit...
4842    oper profit fell eur mn eur mn includ vessel s...
4843    net sale paper segment decreas eur mn second q...
4844    sale finland decreas januari sale outsid finla...
Name: news, Length: 4845, dtype: object

In [9]:
df.isnull().sum()

sentiment    0
news         0
dtype: int64

In [10]:
Le = LabelEncoder()

df.sentiment = Le.fit_transform(df.sentiment)

df.sentiment

0       1
1       0
2       2
3       2
4       2
       ..
4840    0
4841    1
4842    0
4843    0
4844    0
Name: sentiment, Length: 4845, dtype: int32

In [11]:
port_stem = PorterStemmer()

In [12]:
def stemming(news):
  stemming_content = re.sub('[^a-zA-Z]',' ',news)
  stemming_content = stemming_content.lower()
  stemming_content = stemming_content.split()
  stemming_content = [port_stem.stem(word) for word in stemming_content if not word in stopwords.words('english')]
  stemming_content = ' '.join(stemming_content)
  return stemming_content


In [13]:
df.news=df.news.apply(stemming)

In [14]:
df.news

0       technopoli plan develop stage area less squar ...
1       intern electron industri compani elcoteq laid ...
2       new product plant compani would increas capac ...
3       accord compani updat strategi year baswar targ...
4       financ aspocomp growth aspocomp aggress pursu ...
                              ...                        
4840    london marketwatch share price end lower londo...
4841    rinkuskiai beer sale fell per cent million lit...
4842    oper profit fell eur mn eur mn includ vessel s...
4843    net sale paper segment decreas eur mn second q...
4844    sale finland decreas januari sale outsid finla...
Name: news, Length: 4845, dtype: object

In [15]:
x= df.news.values
y=df.sentiment.values

In [16]:
y.ravel()

array([1, 0, 2, ..., 0, 0, 0])

In [17]:
vect=TfidfVectorizer()
vect.fit(x)
x= vect.transform(x)


In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [19]:
y_train

array([1, 2, 0, ..., 1, 1, 2])

In [107]:



model1  = RandomForestClassifier(n_estimators=50,random_state=42)
model1.fit(x_train,y_train)

pred= model1.predict(x_test)



In [108]:
x.shape

(4845, 6678)

In [109]:
score=accuracy_score(pred,y_test)
score

0.7327141382868937

In [110]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Dense(units=512, activation="relu",input_shape=(x_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(units=3)
])

# Compile the model
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])

# Train the model (ensure x_train and y_train are defined)
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10


c:\Users\harsh\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5998 - loss: 0.8891
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8787 - loss: 0.3194
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9630 - loss: 0.1085
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9856 - loss: 0.0465
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9897 - loss: 0.0374
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9895 - loss: 0.0298
Epoch 7/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9933 - loss: 0.0171
Epoch 8/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9906 - loss: 0.0229
Epoch 9/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9914 - loss: 0.0174
Epoch 10/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9947 - loss: 0.0143


In [111]:
pred2 = model.predict(x_test)
pred_categorical = np.argmax(pred2,axis=1)
score =accuracy_score(pred_categorical,y_test)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [112]:
score

0.7048503611971104

In [67]:
import joblib
joblib.dump(model, 'model.pkl')

['model.pkl']

In [95]:
tweets_input = np.array([
    "@BillyM2k 🤣🤣",
    "From the top of Starbase high bay https://t.co/kRtJBrwrqW",
    "RT @cleantechnica: Jay Leno Breaks World Record In Tesla’s New Plaid Model S – “I Love This Car” https://t.co/Rxd7VNIZpx",
    "Stacking Super Heavy Aft Section https://t.co/itydacQ4hM",
    "@MileyCyrus 133T H4X0R got my phone. Nuthin I could do. Sorry babe.",
    "@tobyliiiiiiiii @f_rmnn Cool"
])

df3= pd.DataFrame(data=tweets_input)






In [96]:
df3.columns=['input']

In [97]:
df3

,input
0,@BillyM2k 🤣🤣
1,From the top of Starbase high bay https://t.co...
2,RT @cleantechnica: Jay Leno Breaks World Recor...
3,Stacking Super Heavy Aft Section https://t.co/...
4,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...
5,@tobyliiiiiiiii @f_rmnn Cool


In [98]:
df3.input = df3.input.apply(stemming)

In [99]:
df3.input

0                                             billym k
1              top starbas high bay http co krtjbrwrqw
2    rt cleantechnica jay leno break world record t...
3     stack super heavi aft section http co itydacq hm
4    mileycyru h x r got phone nuthin could sorri babe
5                           tobyliiiiiiiii f rmnn cool
Name: input, dtype: object

In [100]:
x1= vect.fit_transform(df3.input)

In [115]:
pred3 = model.predict(x1)
pred3=np.argmax(pred3,axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [116]:
pred3

array([1, 2, 1, 2, 1, 2], dtype=int64)